In [ ]:
#As we dont have any GPU we will be using google collab for GPU usage
#Mout at drive 
import sys
from google.colab import drive
from pathlib import Path
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

''

In [ ]:
#import packages

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sn
%matplotlib inline
import re
import sys
import warnings
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.corpus import wordnet
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
warnings.filterwarnings("ignore")

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/multi class bangla sentiment analysis/dataset/bangla_comments_tokenized.csv')
df.tail()


,Category,Gender,label,text,stem_text_list
43996,Social,Male,not bully,হিরো আলম এগা যাও,"['হিরো', 'আলম', 'এগা', 'যাও']"
43997,Social,Male,not bully,হিরো আলম সাপোর্ট অসংখ্য ধন্যবাদ আপনা,"['হিরো', 'আলম', 'সাপোর্ট', 'অসংখ্য', 'ধন্যবাদ'..."
43998,Social,Male,not bully,হিরো ভাই এগা য়াও,"['হিরো', 'ভাই', 'এগা', 'য়াও']"
43999,Actor,Female,not bully,হুম ভাও তোমরা এগা যাও তোমা পিছনে আছি,"['হুম', 'ভাও', 'তোমরা', 'এগা', 'যাও', 'তোমা', ..."
44000,Social,Male,not bully,হ্যালো তোমা সাথে আমা চ্যাট করবে,"['হ্যালো', 'তোমা', 'সাথে', 'আমা', 'চ্যাট', 'কর..."


In [ ]:
df['label'] = df['label'].replace({'not bully':'acceptable'})
df.head()

,Category,Gender,label,text,stem_text_list
0,Actor,Female,sexual,হালা পুত মদ খাওয় সময় রাত বেলা মদ খাই দিন বেল...,"['হালা', 'পুত', 'মদ', 'খাওয়', 'সময়', 'রাত', ..."
1,Singer,Male,acceptable,ঘরে শুট কেমন ক্যামেরা ছিল,"['ঘরে', 'শুট', 'কেমন', 'ক্যামে', 'ছিল']"
2,Actor,Female,acceptable,অরে বাবা পাগল,"['অরে', 'বাবা', 'পাগল']"
3,Sports,Male,acceptable,ক্যাপ্ট অফ বাংলাদেশ,"['ক্যাপ্ট', 'অফ', 'বাংলাদেশ']"
4,Politician,Male,troll,পটকা মাছ,"['পটকা', 'মাছ']"


In [ ]:
#categorical Columns OneHotEncoder
from sklearn.base import TransformerMixin,BaseEstimator
from sklearn.preprocessing import OneHotEncoder
class My_encoder(BaseEstimator, TransformerMixin):
   
    def __init__(self,drop = 'first',sparse=False):
        self.encoder = OneHotEncoder(drop = drop,sparse = sparse)
        self.features_to_encode = []
        self.columns = []
    
    def fit(self,X_train,features_to_encode):
        
        data = X_train.copy()
        self.features_to_encode = features_to_encode
        data_to_encode = data[self.features_to_encode]
        self.columns = pd.get_dummies(data_to_encode,drop_first = True).columns
        self.encoder.fit(data_to_encode)
        return self.encoder
    
    def transform(self,X_test):
        
        data = X_test.copy()
        data.reset_index(drop = True,inplace =True)
        data_to_encode = data[self.features_to_encode]
        data_left = data.drop(self.features_to_encode,axis = 1)
        data_encoded = pd.DataFrame(self.encoder.transform(data_to_encode),columns = self.columns)
        
        return pd.concat([data_left,data_encoded],axis = 1)

In [ ]:
#perform encoding
features_to_encode = ['Category','Gender']
#numerical_features=df.drop(features_to_encode,axis=1)
enc = My_encoder()
enc.fit(df,features_to_encode)
df = enc.transform(df)
df.head()

,label,text,stem_text_list,Category_Politician,Category_Singer,Category_Social,Category_Sports,Gender_Male
0,sexual,হালা পুত মদ খাওয় সময় রাত বেলা মদ খাই দিন বেল...,"['হালা', 'পুত', 'মদ', 'খাওয়', 'সময়', 'রাত', ...",0.0,0.0,0.0,0.0,0.0
1,acceptable,ঘরে শুট কেমন ক্যামেরা ছিল,"['ঘরে', 'শুট', 'কেমন', 'ক্যামে', 'ছিল']",0.0,1.0,0.0,0.0,1.0
2,acceptable,অরে বাবা পাগল,"['অরে', 'বাবা', 'পাগল']",0.0,0.0,0.0,0.0,0.0
3,acceptable,ক্যাপ্ট অফ বাংলাদেশ,"['ক্যাপ্ট', 'অফ', 'বাংলাদেশ']",0.0,0.0,0.0,1.0,1.0
4,troll,পটকা মাছ,"['পটকা', 'মাছ']",1.0,0.0,0.0,0.0,1.0


In [ ]:
#fill na values with zero
df = df.fillna(0)
df.head()

,label,text,stem_text_list,Category_Politician,Category_Singer,Category_Social,Category_Sports,Gender_Male
0,sexual,হালা পুত মদ খাওয় সময় রাত বেলা মদ খাই দিন বেল...,"['হালা', 'পুত', 'মদ', 'খাওয়', 'সময়', 'রাত', ...",0.0,0.0,0.0,0.0,0.0
1,acceptable,ঘরে শুট কেমন ক্যামেরা ছিল,"['ঘরে', 'শুট', 'কেমন', 'ক্যামে', 'ছিল']",0.0,1.0,0.0,0.0,1.0
2,acceptable,অরে বাবা পাগল,"['অরে', 'বাবা', 'পাগল']",0.0,0.0,0.0,0.0,0.0
3,acceptable,ক্যাপ্ট অফ বাংলাদেশ,"['ক্যাপ্ট', 'অফ', 'বাংলাদেশ']",0.0,0.0,0.0,1.0,1.0
4,troll,পটকা মাছ,"['পটকা', 'মাছ']",1.0,0.0,0.0,0.0,1.0


In [ ]:
#divide the model for trainning and testing
df.text=df.text.apply(str)
X = df.text
y = df.label
categories = ['label_sexual','label_religious','label_troll','label_threat','label_acceptable']  #targeted labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

#checking...
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)


(30800,)
(13201,)
(30800,)
(13201,)


**MultinomialNB model with Tfidf using pipeline**

In [ ]:
# Define a pipeline combining a text feature extractor with multi lable classifier
TFIDF_NB_pipeline = Pipeline([

                ('tfidf', TfidfVectorizer(tokenizer=lambda x: x.split(),lowercase=False,max_features=6000,min_df=5,ngram_range=(1,2))),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])

TFIDF_NB_pipeline.fit(X_train, y_train)
prediction = TFIDF_NB_pipeline.predict(X_test)

print('accuracy %s' % accuracy_score(prediction, y_test))
print(classification_report(y_test, prediction, target_names=categories))


accuracy 0.7527460040905992
                  precision    recall  f1-score   support

    label_sexual       0.76      0.88      0.81      4580
 label_religious       0.74      0.82      0.78      2204
     label_troll       0.77      0.67      0.72      2682
    label_threat       0.88      0.45      0.59       491
label_acceptable       0.72      0.64      0.68      3244

        accuracy                           0.75     13201
       macro avg       0.78      0.69      0.72     13201
    weighted avg       0.75      0.75      0.75     13201



**SGDclassifier model with TFIDF using pipeline**

In [ ]:

TFIDF_SGD_pipeline = Pipeline([
                               
                ('tfidf', TfidfVectorizer(tokenizer=lambda x: x.split(),lowercase=False,max_features=6000,min_df=5,ngram_range=(1,2))),
                ('clf', OneVsRestClassifier(SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42, max_iter=90, tol=None)))
            ])


TFIDF_SGD_pipeline.fit(X_train, y_train)
prediction = TFIDF_SGD_pipeline.predict(X_test)

print('accuracy %s' % accuracy_score(prediction, y_test))
print(classification_report(y_test, prediction, target_names=categories))


accuracy 0.7996363911824862
                  precision    recall  f1-score   support

    label_sexual       0.74      0.93      0.82      4580
 label_religious       0.92      0.86      0.89      2204
     label_troll       0.84      0.73      0.78      2682
    label_threat       0.86      0.58      0.70       491
label_acceptable       0.79      0.67      0.72      3244

        accuracy                           0.80     13201
       macro avg       0.83      0.75      0.78     13201
    weighted avg       0.81      0.80      0.80     13201



**Logistic Regression model with TFIDF using pipeline**

In [ ]:
TFIDF_LR_pipeline = Pipeline([

                ('tfidf', TfidfVectorizer(tokenizer=lambda x: x.split(),lowercase=False,max_features=6000,min_df=5,ngram_range=(1,2))),
                ('clf', OneVsRestClassifier(LogisticRegression(C=0.7,random_state=42,tol=0.0001,max_iter=200)))
            ])

TFIDF_LR_pipeline.fit(X_train, y_train)
prediction = TFIDF_LR_pipeline.predict(X_test)

print('accuracy %s' % accuracy_score(prediction, y_test))
print(classification_report(y_test, prediction, target_names=categories))



accuracy 0.7875160972653586
                  precision    recall  f1-score   support

    label_sexual       0.74      0.92      0.82      4580
 label_religious       0.91      0.84      0.87      2204
     label_troll       0.83      0.71      0.77      2682
    label_threat       0.87      0.49      0.62       491
label_acceptable       0.75      0.68      0.71      3244

        accuracy                           0.79     13201
       macro avg       0.82      0.73      0.76     13201
    weighted avg       0.79      0.79      0.78     13201



**RandomforrestClassifier model with TFIDF using pipeline**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
TFIDF_DT_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(tokenizer=lambda x: x.split(),lowercase=False,max_features=6000,min_df=5,ngram_range=(1,2))),
                ('clf', OneVsRestClassifier(RandomForestClassifier(n_estimators=200)))
            ])


TFIDF_DT_pipeline.fit(X_train, y_train)
prediction = TFIDF_DT_pipeline.predict(X_test)

print('accuracy %s' % accuracy_score(prediction, y_test))
print(classification_report(y_test, prediction, target_names=categories))




accuracy 0.8034997348685705
                  precision    recall  f1-score   support

    label_sexual       0.77      0.89      0.83      4580
 label_religious       0.90      0.88      0.89      2204
     label_troll       0.83      0.75      0.79      2682
    label_threat       0.81      0.65      0.72       491
label_acceptable       0.76      0.70      0.73      3244

        accuracy                           0.80     13201
       macro avg       0.81      0.77      0.79     13201
    weighted avg       0.81      0.80      0.80     13201

